In [6]:
from gpkit import Model, Variable, units
import gpkit
from numpy import pi
import math
from __future__ import division
gpkit.settings["latex_modelname"] = False

class ltaHALE(Model):
    def setup(self):

        constraints = []

        # Constants
        g = Variable('g', 9.81,'m/s^2', 'gravitational constant')
        R_earth = Variable('R_{earth}', 6371*10^3, 'meters', 'radius of the earth')
        R = Variable('R', 287, 'J/kg*K', 'ideal gas constant')

        # Requirements
        m_pay = Variable('m_{pay}', 10, 'lb', 'mass of payload') 
        Thresh_Diameter = Variable('Thresh_{diam}', 100, 'km', 'threshold footprint diameter')
        Object_Diameter = Variable('Object_{diam}', 300, 'km', 'objective footprint diameter')
        Inclination_Angle = Variable('angle_{incl}', 5, 'deg', 'max antenna inclination angle')
        time = Variable('t', 5*24, 'hr', 'flight time')
        P_payload = Variable('P_{pay}', 10, 'W', 'power requirement of payload')

        # Assumptions
        Cd = Variable('C_d', 0.5, '-', 'drag coefficient') #Sphere
        eta_prop = Variable(r'\eta_{prop}', 0.75, '-', 'propulsive efficiency')
        m_bat = Variable('m_{bat}', 2, 'kg', 'mass of payload battery')
        m_avi = Variable('m_{avi}', 1, 'kg', 'mass of avionics')
        m_fixed = Variable('m_{fixed}', 'lbm', 'fixed mass')

        constraints.extend([m_fixed >= m_pay + m_avi + m_bat])

        f_fixed_pay = Variable('f_{weightMassFraction}', 0.1, '-', 'Fixed weight mass fraction')

        W = Variable('W', 'lbf', 'vehicle weight')
        W_diesel = Variable('W_{diesel}', 'lbf', 'mission diesel fuel weight')
        m_balloon = Variable('m_balloon', 'kg', 'mass of balloon skin')
        vol = Variable('vol', 'm^3', 'vehicle Volume')
        rho_he = Variable(r'\rho_{he}', 0.1786, 'g/L', 'density of helium at stp')
        constraints.extend([W >= m_fixed * g/f_fixed_pay + W_diesel + m_balloon * g + vol * rho_he * g])

#         # # Atmospheric Stuff -- MATLAB syntax
#         T=zeros(1,30000); # Celsius Temperature Matrix 
#         P=zeros(1,30000); # Kilopascal Pressure Matrix
#         for h=1:30000
#             if h<11000
#                 T(1,h)=15.04-0.00649*h;
#                 P(1,h)=101.29*((T(1,h)+273.1)/288.08)^5.256;
#             elseif h<25000 && h>=11000
#                 T(1,h)=-56.46;
#                 P(1,h)=22.65*exp(1.73-0.000157*h);
#             elseif h>=25000
#                 T(1,h)=-131.21+0.00299*h;
#                 P(1,h)=2.488*((T(1,h)+273.1)/216.6)^-11.388;
#             end
#         end

#         p=P./(0.2869.*(T+273.1)); # # kg/m^3 Density Matrix
#         h=1:30000;

#         Altitude of Flight accounting for Curvature of the Earth (Assumed Spherical Earth)
#         Footprint = Variable('D', 'm', 'Diameter of Communications Footprint')
#         Theta = Variable(r'\Theta', Footprint / (4 * pi * R_earth) * 2*math.pi, 'rad', 'theta')
#         Chord_earth = Variable('Chord_{earth}', 2 * R_earth * math.sin(Theta/2), 'm', 'chord length of the earth')
#         Alpha = Variable(r'\Alpha', (math.pi - Theta)/2, 'deg', 'alpha')
#         Beta = Variable(r'\Beta', math.pi - Alpha, 'rad', 'beta')
#         Zeta = Variable(r'\Zeta', math.pi/2 - Alpha, 'rad', 'zeta')
#         Gamma = Variable(r'\Gamma', math.pi - Beta - Inclination_Angle - Zeta, 'rad', 'gamma')
        

#         constraints.extend([Footprint>=Object_Diameter, 
#                 H >= Chord_earth * math.sin(Inclination_Angle + Zeta)/math.sin(Gamma)]); # # meters

#         # # Environment
        rho = Variable(r'\rho', 'kg/m^3', 'air density')
#         press = P(1,H); # pressure for this selected altitude
#         temp = T(1,H); # # temperature for this selected altitude
        h = Variable("h", "ft", "Altitude")
        vel = Variable('vel', 'm/s', 'cruise velocity')
        v_wind = Variable('v_{wind}',35, 'm/s', 'wind velocity')
        

        constraints.extend([vel >= v_wind])
        
#         constraints.extend([v_wind>=-0.175*h**2+4.25*h+5])  upside down parabola model of wind with altitude. 

#         # # LTA Sizing
#         vol = Variable('vol', 'm^3', 'vehicle Volume')
        t = Variable('thickness ratio', 0.5, '-', 'thickness ratio')
        c = Variable('chord', 'm', 'length of vehicle')
        constraints.extend([c >= (vol/(10 * pi * t * (2/3 * 0.2969 - 0.126 * 0.5 - 0.3516 * 1/3 + 0.2843 * 1/4 - 0.1015 * 1/5)))**(1/3)])
#         c = Variable('chord', (Vol/(10 * pi * t * (2/3 * 0.2969 - 0.126 * 0.5 - 0.3516 * 1/3 + 0.2843 * 1/4 - 0.1015 * 1/5)))^(1/3), 'm', 'chord length')
        area_front = Variable('area_{front}', 'm^2', 'frontal area')
        area_surface = Variable('area_{surface}', 'm^2', 'surface area')
        r = Variable('r', 'ft', 'vehicle radius')
#         rho_he = Variable(r'\rho_{he}', 0.1786, 'g/L', 'density of helium at stp')
        rho_h = Variable(r'\rho_{h}', 0.08988, 'g/L', 'density of hydrogen at stp')
        t_skin = Variable('t_{skin}', 0.125, 'in', 'thickness of skin of balloon')
        rho_skin = Variable(r'\rho_{skin}', 1.45, 'g/cm^3', 'density of polyester skin')
        constraints.extend([vol >= W/(rho * g),
                            r == t * c/2,
                            area_front == pi * r**2,
                            area_surface == 4 * pi * r**2,
                            m_balloon == t_skin* area_surface * rho_skin])

        # Power Consumption
        P_shaft = Variable('P_{shaft}', 'W', 'Shaft power')
        constraints.extend([P_shaft >= 1 /eta_prop/2. * rho * vel**3 * Cd * area_front])

        E_total = Variable('E_{total}', "MJ", "total mission energy")

#         # Carbon Fuels
        h_diesel = Variable('h_{diesel}', 35.86, 'MJ/L', 'diesel energy density')
        Diesel_eff = Variable(r'\Eta_{diesel}', 0.5, '-', 'diesel engine efficiency')
        Diesel_dens = Variable(r'\Rho_{diesel}', 0.832, 'kg/L', 'diesel density')
        constraints.extend([W_diesel >= E_total/h_diesel*Diesel_dens*g,
                            E_total >= (P_payload + P_shaft)*time])
#         Gasoline_Edens = Variable('gasoline_{edens}', 32.4, 'MJ/L', 'gasoline energy density')
#         Gasoline_eff = Variable(r'\Eta_{gasoline}', 0.25, '-', 'gasoline engine efficiency')
#         Gasoline_dens = Variable(r'\Rho_{gasoline}', 0.75, 'kg/L', 'gasoline density')
#         Jetfuel_Edens = Variable('jetfuel_{edens}', 37.4, 'MJ/L', 'jetfuel energy density')
#         Jetfuel_dens = Variable(r'\Rho_{jetfuel}', 0.81, 'kg/L', 'jetfuel density')
#         constraints.extend([Diesel_needed >= E_total_joules/(Diesel_Edens * 10^6),
#                             Gasoline_needed >= E_total_joules/(Gasoline_Edens * 10^6),
#                             Jetfuel_needed >= E_total_joules/(Jetfuel_Edens * 10^6),
#                             Diesel_mass >= Diesel_needed*Diesel_dens,
#                             Gasoline_mass >= Gasoline_needed * Gasoline_dens,
#                             Jetfuel_mass >= Jetfuel_needed * Jetfuel_dens,
#                             Diesel_fraction >= Diesel_mass/(W/g),
#                             Gasoline_fraction >= Gasoline_mass/(W/g),
#                             Jetfuel_fraction >= Jetfuel_mass/(W/g)])

        # Atmosphere model
        
        p_sl = Variable("p_{sl}", 101325, "Pa", "Pressure at sea level")
        T_sl = Variable("T_{sl}", 288.15, "K", "Temperature at sea level")
        L_atm = Variable("L_{atm}", 0.0065, "K/m", "Temperature lapse rate")
        T_atm = Variable("T_{atm}", "K", "air temperature")
        M_atm = Variable("M_{atm}", 0.0289644, "kg/mol", "Molar mass of dry air")
        R_atm = Variable("R_{atm}", 8.31447, "J/mol/K")
        TH = (g*M_atm/R_atm/L_atm).value.magnitude  # dimensionless
        constraints.extend([h <= 20000*units.m,  # Model valid to top of troposphere
                            h >=14900*units.m, #for footprint
                            T_sl >= T_atm + L_atm*h,     # Temp decreases w/ altitude
                            rho <= p_sl*T_atm**(TH-1)*M_atm/R_atm/(T_sl**TH)])  # http://en.wikipedia.org/wiki/Density_of_air#Altitude
        
        # station keeping requirement
        footprint = Variable("d_{footprint}", 300, 'km', "station keeping footprint diameter")
        lu = Variable(r"\theta_{look-up}", 5, '-', "look up angle")
        R_earth = Variable("R_{earth}", 6371, "km", "Radius of earth")
        tan_lu = lu*pi/180. + (lu*pi/180.)**3/3.  # Taylor series expansion
        # approximate earth curvature penalty as distance^2/(2*Re)
        constraints.extend([h >= tan_lu*0.5*footprint + footprint**2/8./R_earth])
        
        objective = W

        return objective, constraints

m = ltaHALE()
m.solve('cvxopt')


Using solver 'cvxopt'
Solving for 15 variables.
Solving took 0.0333 seconds.

Cost
----
 8.14e+06 [lbf] 

Free Variables
--------------
     E_{total} : 1.65e+07   [MJ]      total mission energy      
     P_{shaft} : 3.818e+07  [W]       Shaft power               
       T_{atm} : 191.3      [K]       air temperature           
             W : 8.14e+06   [lbf]     vehicle weight            
    W_{diesel} : 8.44e+05   [lbf]     mission diesel fuel weight
          \rho : 0.2141     [kg/m**3] air density               
  area_{front} : 1.248e+04  [m**2]    frontal area              
area_{surface} : 4.991e+04  [m**2]    surface area              
         chord : 252.1      [m]       length of vehicle         
             h : 4.888e+04  [ft]      Altitude                  
     m_balloon : 2.298e+05  [kg]      mass of balloon skin      
     m_{fixed} : 16.61      [lb]      fixed mass                
             r : 206.8      [ft]      vehicle radius            
           vel : 35

{'constants': {M_{atm}_ltaHALE2: array(0.0289644),
  R_{atm}_ltaHALE2: array(8.31447),
  \theta_{look-up}_ltaHALE2: array(5),
  d_{footprint}_ltaHALE2: array(300),
  R_{earth}_ltaHALE2: array(6371),
  h_{diesel}_ltaHALE2: array(35.86),
  \Rho_{diesel}_ltaHALE2: array(0.832),
  T_{sl}_ltaHALE2: array(288.15),
  p_{sl}_ltaHALE2: array(101325),
  L_{atm}_ltaHALE2: array(0.0065),
  t_ltaHALE2: array(120),
  P_{pay}_ltaHALE2: array(10),
  C_d_ltaHALE2: array(0.5),
  \eta_{prop}_ltaHALE2: array(0.75),
  m_{bat}_ltaHALE2: array(2),
  m_{avi}_ltaHALE2: array(1),
  f_{weightMassFraction}_ltaHALE2: array(0.1),
  \rho_{he}_ltaHALE2: array(0.1786),
  thickness ratio_ltaHALE2: array(0.5),
  t_{skin}_ltaHALE2: array(0.125),
  \rho_{skin}_ltaHALE2: array(1.45),
  g_ltaHALE2: array(9.81),
  v_{wind}_ltaHALE2: array(35),
  m_{pay}_ltaHALE2: array(10)},
 'cost': array(8140358.392052104),
 'freevariables': {T_{atm}_ltaHALE2: array(191.30000001388385),
  vel_ltaHALE2: array(34.999999995496076),
  h_ltaHAL